In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as f
import torch.optim as optim
import numpy as np
import random
import gym
env = gym.make("CartPole-v1")
import matplotlib.pyplot as plt


/usr/local/lib/python3.9/dist-packages/gym/core.py:317: DeprecationWarning: WARN: Initializing wrapper in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(
/usr/local/lib/python3.9/dist-packages/gym/wrappers/step_api_compatibility.py:39: DeprecationWarning: WARN: Initializing environment in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(


In [ ]:
class Buffer:
  def __init__(self,buffer_size,batch_size):
    self.buffer_size = buffer_size
    self.batch_size = batch_size
    self.states      = []
    self.next_states = []
    self.rewards     = []
    self.terminals   = []
    self.actions     = []
    self.cntr        = 0
    self.clear()
  def store(self,state,next_state,reward,terminal,action):
    self.states.append(state)
    self.next_states.append(next_state)
    self.rewards.append(reward)
    self.terminals.append(terminal)
    self.actions.append(action)
    self.cntr +=1
  def sample(self):
    index  = random.sample(range(len(self.states)),self.batch_size)
    states = np.array(self.states)[index] 
    next_states = np.array(self.next_states)[index]
    rewards     = np.array(self.rewards)[index]
    terminals   = np.array(self.terminals)[index]
    return states ,next_states,rewards,terminals
  def clear(self):
    if self.cntr == self.buffer_size:
      self.states      = []
      self.next_states = []
      self.rewards     = []
      self.terminals   = []
      self.actions     = []
      self.cntr        = 0

In [ ]:
class Network(nn.Module):
  def __init__(self,state_size,action_size):
    super(Network,self).__init__()
    self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    self.state_size = state_size
    self.action_size = action_size
    self.lin1 = nn.Linear(4,64)
    self.lin2 = nn.Linear(64,128)
    self.lin3 = nn.Linear(128,64)
    self.lin4 = nn.Linear(64,self.action_size)
    self.to(self.device)
  def forward(self,x):
    x = f.relu(self.lin1(x))
    x = f.relu(self.lin2(x))
    x = f.relu(self.lin3(x))
    x = self.lin4(x)
    return x

In [ ]:
class Agent:
  def __init__(self,state_size,action_size,buffer_size,batch_size,episodes,steps):
    self.state_size  = state_size
    self.action_size = action_size 
    self.buffer_size = buffer_size
    self.batch_size  = batch_size
    self.episodes    = episodes
    self.steps       = steps
    self.epislon     = 1
    self.gamma       = 0.99
    self.epislon_end = 0.01
    self.epislon_dec = 100000
    self.lr          = 0.00005
    self.episode     = []
    self.loss        = []
    self.device      = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    self.buffer      = Buffer(self.buffer_size,self.batch_size) 
    self.q_net       = Network(self.state_size,self.action_size)
    self.tar_net     = Network(self.state_size,self.action_size)
    self.optim       = optim.Adam(self.q_net.parameters() ,lr = self.lr)
    self.tar_net.load_state_dict(self.q_net.state_dict())
    self.mean_loss   = []
    self.mean_epi    = []
    self.mean_reward = []
    self.rewards     = []
  def save(self):
    torch.save(self.q_net.state_dict(),"DQNq_net.pth")
  def load(self):
    self.q_net.load_state_dict(torch.load("DQNq_net.pth"))
  def choose_actions(self,state):
    state = torch.tensor(state,dtype = torch.float32).to(self.device)
    if random.random() <= self.epislon:
      action =  env.action_space.sample()
      action =  torch.tensor(action)
    else:
      q_values = self.q_net(state)
      action   = q_values[torch.argmax(q_values).item()]
    return action.item()
  def learn(self):
    if len(self.buffer.states) < self.batch_size:
      self.loss.append(0)
      return 
    else:
      states , next_states, rewards , terminals = self.buffer.sample()
      states = torch.tensor(states,dtype = torch.float32).to(self.device)
      next_states = torch.tensor(next_states,dtype = torch.float32).to(self.device)
      rewards = torch.tensor(rewards,dtype = torch.float32).to(self.device)
      q_value = self.q_net(states)
      q_tar   = self.tar_net(next_states)
      y       = sum(rewards) + self.gamma * q_value
      loss    = torch.tensor((y-q_value)**2,requires_grad = True)
      loss    = loss.mean()
      self.loss.append(loss.detach().numpy())
      self.save()
      self.load()
      self.optim.zero_grad()
      loss.backward()
      self.optim.step()
  def ploting(self):
    plt.plot(self.mean_epi,self.mean_loss)
    plt.xlabel("count")
    plt.ylabel("loss")
    plt.savefig("DQN\cartpole.png")
    plt.close()

  def train(self):
    count = 0
    for i in range(self.episodes):
      state = env.reset()
      for j in range(self.steps):
        action = self.choose_actions(state)
        next_state,reward,done,info = env.step(action)
        self.buffer.store(state,next_state,reward,done,action)
        self.episode.append(count)
        self.rewards.append(reward)
        if done:
          self.learn()
          state = next_state
        else:
          self.learn()
          state = next_state
          self.ploting()
        count +=1
      self.mean_loss.append(sum(self.loss)/len(self.loss))
      self.mean_epi.append(i)
      self.mean_reward.append(sum(self.rewards)/len(self.rewards))
      print(f'episode : {i} / {self.episodes} ; reward : {self.mean_reward[-1]} ; loss : {self.mean_loss[-1]}')
      self.loss,self.rewards = [],[]
  def test(self):
    pass

In [ ]:
Agent = Agent(env.observation_space.shape[0],env.action_space.n,10000,32,10000,350)

In [ ]:
Agent.train()

<ipython-input-81-c64ef2e6c74c>:51: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  loss    = torch.tensor((y-q_value)**2,requires_grad = True)


episode : 0 / 10000 ; reward : 0.03428571428571429 ; loss : 13.204269672330959
episode : 1 / 10000 ; reward : 0.08571428571428572 ; loss : 5.8883430613086745
episode : 2 / 10000 ; reward : 0.12285714285714286 ; loss : 12.312359430183344
episode : 3 / 10000 ; reward : 0.05142857142857143 ; loss : 9.960888735083396
episode : 4 / 10000 ; reward : 0.13428571428571429 ; loss : 11.202578119274614
episode : 5 / 10000 ; reward : 0.054285714285714284 ; loss : 11.960983573008228
episode : 6 / 10000 ; reward : 0.04857142857142857 ; loss : 8.92586018075373
episode : 7 / 10000 ; reward : 0.05714285714285714 ; loss : 8.945452559394576
episode : 8 / 10000 ; reward : 0.06285714285714286 ; loss : 8.070580499122794
episode : 9 / 10000 ; reward : 0.04285714285714286 ; loss : 8.085532299329726
episode : 10 / 10000 ; reward : 0.04857142857142857 ; loss : 7.227414149545371
episode : 11 / 10000 ; reward : 0.08857142857142856 ; loss : 7.767562110056922
episode : 12 / 10000 ; reward : 0.025714285714285714 ; lo